# Análisis de datos exploratorio

- Se debe ejecutar despúes del notebook `data.ipynb`
- Este notebook crea el archivo necesario para `export.ipynb`
- Los archivos generados son:
  - `model.h5`: Modelo de red neuronal (Este archivo es necesario para predecir las new entrantes la app web `./client`)
- Es necesario [descargar](https://www.kaggle.com/datasets/danielwillgeorge/glove6b100dtxt) el modelo de Glove de 100 dimensiones.

## Cargar datos

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("features.csv")

In [3]:
df.head()

,features,label
0,win court lose ground uncertainty cloud vote r...,1
1,obama name first africanamerican woman librari...,1
2,daughter allege iowa cop killer state father s...,0
3,burr nunes step aside protect russia probe inv...,1
4,postseason bucs top yank fall tiger split six ...,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76524 entries, 0 to 76523
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   features  76524 non-null  object
 1   label     76524 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


## Cargando el modelo de Word Embedding (Glove 100d)

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [6]:
# Cargar el modelo de glove
def load_glove_model(file_path):
    """Cargar un modelo GloVe desde un archivo de texto."""
    model = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        # Parsear el archivo .txt
        for line in file:
            values = line.split()
            word = values[0]
            vector = [float(value) for value in values[1:]]
            model[word] = vector
    return model

In [7]:
def cosine_similarity(vec1, vec2):
    """Calcular la similitud coseno entre dos vectores."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

def most_similar_words(word, model, top_n=5):
    """Encontrar las palabras más similares a la palabra dada en el modelo."""
    if word not in model:
        print(f"La palabra '{word}' no está en el modelo.")
        return []

    word_vector = model[word]
    similarities = []

    for other_word, other_vector in model.items():
        if other_word != word:
            similarity = cosine_similarity(word_vector, other_vector)
            similarities.append((other_word, similarity))

    # Ordenar por similitud descendente
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Devolver las palabras más similares (top_n)
    return similarities[:top_n]

In [8]:
# Ruta al archivo GloVe
glove_file_path = 'glove.6B.100d.txt'

# Cargar el modelo GloVe
embedding_index = load_glove_model(glove_file_path)

# Ejemplo de uso con la palabra 'ate'
word_to_lookup = 'ate'
similar_words = most_similar_words(word_to_lookup, embedding_index, top_n=10)

print(f"Palabras más similares a '{word_to_lookup}':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity}")

Palabras más similares a 'ate':
eaten: 0.809029408004442
drank: 0.7871671568844296
eat: 0.7813820363677394
eating: 0.7338559174177061
eats: 0.7251667659830515
slept: 0.7152498971261759
smoked: 0.6635120852825582
meal: 0.654029261267389
cooked: 0.6502910720528902
dined: 0.6021937327293632


## Creacion del Word Embedding en base a nuestros datos

In [20]:
# Variables para el modelo
vocab_size = 20000 # Número máximo de palabras a utilizar
max_sequence_length = 300 # Número máximo de palabras en una secuencia
embedding_dim = 100 # Dimensión del vector de embedding

In [21]:
# Tokenizar la data (BOW)
tokenizer = Tokenizer(num_words=vocab_size)  # Instanciamos el tokenizer
tokenizer.fit_on_texts(df["features"])  # convert to string type

# Obtenemos solo las primeras 10000 palabras
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= vocab_size}
word_index = tokenizer.word_index

# Observamos el tamaño del vocabulario
# vocab_size = len(word_index)
# vocab_size

In [24]:
import json

# Invertimos el vocabulario
inv_map = {v: k for k, v in tokenizer.index_word.items()}

# Guarda el vocabulario invertido en un archivo JSON
with open('vocab.json', 'w') as archivo:
    json.dump(inv_map, archivo)

# El vocabulario es guardado para posteriormente ser usado en clasificacion usando web scrapping
print(f'Vocabulario invertido guardado en `vocab.json`')

Vocabulario invertido guardado en `vocab.json`


In [25]:
# Convertimos las palabras en secuencias de números
sequences = tokenizer.texts_to_sequences(df["features"])

# Aplicamos padding a las secuencias
padded_seq = pad_sequences(
    sequences,
    maxlen=max_sequence_length,
    padding="post",
    truncating="post",
)

In [26]:
# Crear el embedding matrix de nuestro dataset
embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))
for word, i in word_index.items():
    if i <= vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [27]:
embedding_matrix[-1]

array([-0.70878 , -0.038324, -0.91316 ,  0.46671 ,  0.4833  ,  1.0924  ,
        0.33082 ,  0.54534 ,  0.68029 , -0.16311 , -0.4366  , -0.051576,
        0.84049 ,  0.14467 , -0.12662 , -0.10113 , -0.13787 , -0.18248 ,
       -0.067652, -0.37398 ,  1.2349  , -0.047318,  0.45152 ,  0.63501 ,
        0.36439 , -0.79032 , -0.47052 ,  0.14723 ,  0.11583 ,  0.48238 ,
       -0.11306 ,  0.079177, -0.35294 ,  0.038699, -0.15867 ,  0.42614 ,
       -0.71546 , -0.87426 , -0.54662 , -0.14163 , -0.10158 ,  0.9077  ,
       -0.95376 ,  0.52829 ,  0.26935 , -0.41898 ,  0.66448 , -0.33511 ,
        0.49773 , -0.91425 ,  0.059091,  0.015071,  0.14594 ,  0.8844  ,
        0.18616 , -0.27119 , -0.37574 , -0.46423 , -0.40642 ,  0.39076 ,
       -0.04388 ,  0.98766 , -0.14428 , -0.089652,  0.06662 ,  0.063583,
        0.74573 , -0.60276 ,  0.19722 ,  0.68743 ,  0.20701 , -0.50921 ,
        0.4956  , -0.065256, -0.1679  , -0.30023 , -0.14353 , -0.37801 ,
        0.055353,  0.10654 ,  0.21028 , -0.60749 , 

In [35]:
padded_seq[6]

array([  218,   514,   339,  3658,     9,    23,   968,   373,   323,
        1058,    22,   795,     9,   331,   212,   743,    53,     3,
          92,     5,   388,  2355,  1572,  1197,   339,  3658,   339,
         150,   220,  1028,  2849,   331,    31,  4551,  3401,  1940,
         115,   144,  3658,  4127,    83,   339,   561,  2970,  1509,
        1028,   115,   476,  2460,  3866,   956,  2450,    31,  1986,
       15505,    31,  7476,  2450,  4551,  3401,  4733,   193,  1940,
        2292,   164,     6,   285,   432,   772,   129,   609,    51,
         153,  1544,  2947,  9658,  5947,  2450,   659, 16854,   331,
         157,  1052,   877,     3,   339,  4616,   922,   331,   115,
        1199,   124,    59,   445,   331,   180,   566,   436,  2500,
        2849,  2450,   198,  1719,    16,   198,  1575,  9658,  2272,
       16279,   331,   956,  3447,   544, 14577, 10081,   614,  2450,
       15505,   705,   301,    77,  2027,    46,  1028,   617,  1407,
         609,   129,

# Entrenar el modelo

In [36]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en train (80%) y test (20%) 
X_train, X_test, y_train, y_test = train_test_split(padded_seq, df['label'], test_size=0.20, random_state=42, stratify=df['label'])

## Redes Neuronales

In [37]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding, LSTM

In [38]:
# Construir y entrenar la red neuronal
model = Sequential()

# Capa de Embedding
model.add(
    Embedding(
        input_dim=vocab_size + 1,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=max_sequence_length,
        trainable=False,
    )
)
model.add(LSTM(units=100, return_sequences=True, dropout=0.2))
model.add(LSTM(units=65, return_sequences=True, dropout=0.2))
model.add(LSTM(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          2000100   
                                                                 
 lstm (LSTM)                 (None, 300, 100)          80400     
                                                                 
 lstm_1 (LSTM)               (None, 300, 65)           43160     
                                                                 
 lstm_2 (LSTM)               (None, 32)                12544     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2136237 (8.15 MB)
Trainable params: 136137 (531.79 KB)
Non-trainable params: 2000100 (7.63 MB)
_________________________________________________________________


In [39]:
history = model.fit(
    np.array(X_train),
    y_train,
    epochs=5,
    batch_size=32,
    verbose=True,
    validation_data=(np.array(X_test), y_test),
    workers=4
)

Epoch 1/5
 214/1914 [==>...........................] - ETA: 16:39 - loss: 0.6521 - accuracy: 0.6189

In [ ]:
# Guardar el modelo
model.save("modelo.h5")

# Ver importancia de palabras

In [ ]:
%pip install lime

In [ ]:
from lime.lime_text import LimeTextExplainer

class_names=['NotFake','IsFake']
explainer= LimeTextExplainer(class_names=class_names)

def predict_proba(data):
  list_tokenized_ex = tokenizer.texts_to_sequences(data)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_sequence_length)
  pred=model.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp]))
  return np.array(returnable)

In [ ]:
print("Actual rating",df['label'][10])
explainer.explain_instance(df['features'][10],predict_proba).show_in_notebook(text=True)

In [ ]:
df["features"].describe()

In [ ]:
df.isnull().sum()

## Frecuencia de Palabras

In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Bolsa de palabras
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(df["features"])

# Obtenemos las palabras y las frecuencias
palabras = vectorizer.get_feature_names_out()
frecuencias = texto_features.toarray()

In [ ]:
# Sumar las frecuencias de todas las palabras
frecuencias_totales = frecuencias.sum(axis=0)

# Obtener las palabras más frecuentes y sus frecuencias
palabras_mas_frecuentes = [palabras[i] for i in frecuencias_totales.argsort()[::-1][:10]]
frecuencias_mas_frecuentes = [frecuencias_totales[i] for i in frecuencias_totales.argsort()[::-1][:10]]

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 6))
plt.barh(palabras_mas_frecuentes, frecuencias_mas_frecuentes, color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('Palabra')
plt.title('Palabras más frecuentes')
plt.gca().invert_yaxis()  # Invertir el eje y para mostrar las palabras más frecuentes arriba
plt.show()

# Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(np.array(X_test), y_test)
print(f'Accuracy: {accuracy}')

In [ ]:
# Predecir con el modelo
predictions = model.predict(np.array(X_test))

# Convertir las probabilidades en clases
predictions = list(map(lambda x: 1 if (x > 0.5) else 0, predictions))

# Mostrar el reporte de clasificación
cm = confusion_matrix(y_test, predictions, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot()

In [ ]:
# Mostrar el reporte de clasificación
print(classification_report(y_test, list(predictions), digits=4))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()